In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import pandas as pd
import numpy as np

import  matplotlib.pyplot as plt
import pickle

from chemistry import Molecule
from util import score
from pprint import pprint


In [3]:
with open('../data/molecules_enh.pickle', 'rb') as f:
    molecules = pickle.load(f)

In [4]:
structures = pd.read_feather('../data/structures_enh.feather')
molecules_df = pd.read_feather('../data/molecules.feather')
labelled = pd.read_feather('../data/train.feather')
unlabelled = pd.read_feather('../data/test.feather')

/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning:

.labels was deprecated in version 0.24.0. Use .codes instead.



In [5]:
dipole_moments = pd.read_feather('../data/dipole_moments.feather')
magnetic_shielding_tensors = pd.read_feather('../data/magnetic_shielding_tensors.feather')
mulliken_charges = pd.read_feather('../data/mulliken_charges.feather')
potential_energy = pd.read_feather('../data/potential_energy.feather')
scalar_coupling_contributions = pd.read_feather('../data/scalar_coupling_contributions.feather')

In [6]:
labelled_enh = labelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
#labelled_enh.head(32)

In [7]:
unlabelled_enh = unlabelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
#unlabelled_enh.head(32)

In [8]:
len(labelled), len(labelled_enh)

(4658147, 4658147)

In [9]:
nn_args = dict(dipole_moments=dipole_moments,
               magnetic_shielding_tensors=magnetic_shielding_tensors,
               mulliken_charges=mulliken_charges,
               potential_energy=potential_energy,
               scalar_coupling_contributions=scalar_coupling_contributions)

In [ ]:
model.structures.dtypes

In [ ]:
labelled_enh.dtypes

In [ ]:
from models import NNModel, partition_data

print('Running')

#data_df = labelled[labelled.type == '1JHC'].head(10)
data_df = labelled_enh.head(10)
train_df, test_df = partition_data(data_df, train_frac=1)
train_df = train_df.copy()
        
model = NNModel(model_args = dict(molecules=molecules,
                                  structures=structures),
               **nn_args)
#model.corr(train_df, train_df)
model.setup_data(train_df, train_df)

In [ ]:
list(model.input_df.columns)

In [ ]:
model.input_df.dtypes

In [ ]:
labelled_enh.head()

In [ ]:
import itertools

In [ ]:
df = labelled_enh.head(32)

In [ ]:
#df[['molecule_name']].merge(dipole_moments, left_on='molecule_name', right_on='molecule_name')


In [10]:
def plot_history(history, label):
    n = 5
    
    loss = np.array(history.history['scc_loss'], dtype='float32')
    val_loss = np.array(history.history['val_scc_loss'], dtype='float32')
    
    loss_mean = loss.mean()
    loss_nstd = n * loss.std()
    
    val_loss_mean = val_loss.mean()
    val_loss_nstd = n * val_loss.std()
       
#     mn = min(loss_mean - loss_nstd, val_loss_mean - val_loss_nstd)
#     mx = min(loss_mean + loss_nstd, val_loss_mean + val_loss_nstd)
    mn = loss_mean - loss_nstd
    mx = loss_mean + loss_nstd
    
#     print(loss.min(), loss.max())
#     print(val_loss.min(), val_loss.max())
#     print(loss_mean, loss_nstd, val_loss_mean, val_loss_nstd)
#     print(mn, mx)
    
    plt.plot(loss)
    plt.plot(val_loss)
    plt.title('Loss for %s' % label)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.ylim(mn, mx)
    _= plt.legend(['Train','Validation'], loc='upper left')

In [11]:
nn_args = dict(dipole_moments=dipole_moments,
               magnetic_shielding_tensors=magnetic_shielding_tensors,
               mulliken_charges=mulliken_charges,
               potential_energy=potential_energy,
               scalar_coupling_contributions=scalar_coupling_contributions,
               epochs=200,
               learning_rate=0.001,
               validation_split=0.3
               )

In [18]:
### from models import NNModel
from models import partition_data, NNModel

# import tensorflow as tf
# from tensorflow.keras import backend as tfbe

# tf.reset_default_graph()

# atom_count_ranges = [(1, 10), (11, 15), (16, 20), (21, 25), (26, 30)]
# C_count_ranges = [(1, 5), (6, 6), (7, 7), (8, 9)]
#coupling_types = sorted(labelled_enh.type.unique())[0:1]
coupling_types = ['3JHH']
def test(data, count=5000):
    global model
    global history
    
    plt.figure(figsize=(25, 25))
    
    for i, t in enumerate(coupling_types, 1):    
        data_df = data[data.type == t].head(count)

        train_df, valid_df, test_df = partition_data(data_df, train_frac=0.7, valid_frac=0)

        if len(train_df) < 10 or len(test_df) < 10:
            continue

        print(f'Training {len(train_df)} samples for {t}')

        model = NNModel(dict(molecules=molecules, 
                             structures=structures),
                        **nn_args)
        history = model.fit(train_df, train_df)
        
        print(f'Evaluating {len(test_df)} samples')
        output, score = model.evaluate(test_df, test_df)
        print(f'{t} score: {score} (trained on {len(train_df)} elements)')


        plt.subplot(4, 4, 2 * i - 1)
        plot_history(history, t)
        
        #print(output.shape, test_df.values.shape)
        #pprint(list(zip(output[0:20, 0], test_df.scalar_coupling_constant[0:20])))

        plt.subplot(4, 4, 2 * i)
        plt.plot(test_df.scalar_coupling_constant, output, '*')
        mn = min(test_df.scalar_coupling_constant.min(), output.min())
        mx = min(test_df.scalar_coupling_constant.max(), output.max())

        plt.plot([mn, mx], [mn, mx])
        plt.title(t)
                        
    plt.show()
        
test(labelled_enh, 30000)


Training 21000 samples for 3JHH
  Setting up data


/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



  Creating model
  Fitting model
Train on 14699 samples, validate on 6301 samples
Epoch 1/200


ResourceExhaustedError: OOM when allocating tensor with shape[2048,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node 1024x11_7_dense_2/kernel/Initializer/random_normal/RandomStandardNormal (defined at /home/gautham/work/kaggle/scalar_coupling/notebooks/models.py:759) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for '1024x11_7_dense_2/kernel/Initializer/random_normal/RandomStandardNormal':
  File "/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/

<Figure size 1800x1800 with 0 Axes>

In [ ]:
model.last_numeric_input_df.columns[np.where(np.isinf(model.last_numeric_input_df.values))[1]]

In [ ]:
w, b = model.model.layers[1].get_weights()

In [ ]:
w2 = w.sum(axis=1)

In [ ]:
w2.min(), w2.max(), w2.mean()

In [ ]:
model.numeric_input_df.columns[np.abs(w.sum(axis=1)) > 10]

In [ ]:
from models import partition_data, NNModel

def train(data):
    models = {}
    for t in sorted(data.type.unique()):
        train_df = data[data.type == t]
        print(f'Training {len(train_df)} samples for {t}')
        model = NNModel(dict(molecules=molecules, 
                             structures=structures),
                        **nn_args)
        models[t] = model
        model.fit(train_df, train_df)
    
    return models

models = train(labelled_enh)

In [ ]:
unlabelled_enh.shape

In [ ]:
from models import NNModel

print('Copying')
new_models = {}
for name in models:
    new_models[name] = NNModel(dict(molecules=molecules, 
                                    structures=structures),
                               **nn_args)
    new_models[name].model = models[name].model
    new_models[name].input_scaler = models[name].input_scaler
    new_models[name].output_scaler = models[name].output_scaler
    

In [ ]:
from models import NNModel

def predict(data, models):
    out_df = None
    
    for t in sorted(data.type.unique()):
        predict_df = data[data.type == t]
        print(f'Predicting {len(predict_df)} samples for {t}')
        output = models[t].predict(predict_df)
        
        id = predict_df['id']
        out_df_coupling = pd.DataFrame(data={'id':id, 'scalar_coupling_constant':output.flatten()}, index=predict_df.index)
        
        if out_df is None:
            out_df = out_df_coupling
        else:
            out_df = out_df.append(out_df_coupling).sort_index()

    return out_df.sort_index()
    
#%prun -s cumulative f(unlabelled.head(10000))
prediction = predict(unlabelled_enh, new_models)

In [ ]:
prediction.head()

In [ ]:
prediction.to_csv('../data/pred.csv',index=False)

In [ ]:
import pickle
with open('nn.pickle', 'wb') as f:
    pickle.dump(models, f)

In [ ]:
models

In [ ]:
import time
time.strptime()